In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import scipy
from astropy.table import Table, join
import illustris_python as il
from astropy.table import setdiff
import time as time
import scipy.signal as signal
import scipy.interpolate as interpolate
from astropy.cosmology import Planck15 as cosmo
from astropy.table import vstack
import h5py
import matplotlib.gridspec as gridspec

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:90% !important; }</style>"))

In [15]:
"""
300-1 Loading
"""
basePath = '/home/mattbel/Desktop/Research/BhuvSummer2020/Illustris-300-1/output'
snapshot_ind = 50


#Header gives basic group info
header = il.groupcat.loadHeader(basePath, snapshot_ind)
scale_factor = header['Time']

#Relevant fields
fields = ['SubhaloFlag','SubhaloGrNr', 'SubhaloMass', 'SubhaloMassType', 'SubhaloPos','SubhaloSFR',
          'SubhaloSpin','SubhaloStarMetallicity','SubhaloStellarPhotometrics','SubhaloVel','SubhaloVmax', 'SubhaloBHMass','SubhaloStarMetalFractions']
fields2 = ['GroupFirstSub', 'GroupMass', 'GroupMassType','GroupNsubs','GroupPos','GroupVel',
           'Group_M_Mean200','Group_M_Crit200','Group_M_Crit500','Group_R_Crit200','Group_R_Crit500','Group_R_Mean200']
#Load in the subhalo/halo tables
subhalo_grp = il.groupcat.loadSubhalos(basePath, snapshot_ind, fields = fields)
halo_grp = il.groupcat.loadHalos(basePath, snapshot_ind, fields = fields2)


#Get the keys for each
subhalo_keys = (list(subhalo_grp.keys()))
halo_keys = (list(halo_grp.keys()))

print(subhalo_grp['count'])
print(halo_grp['count'])
halo_grp.pop('count')
halo_keys.remove('count')
subhalo_grp.pop('count')
subhalo_keys.remove('count')

subhalo_keys = (list(subhalo_grp.keys()))
halo_keys = (list(halo_grp.keys()))
subhalo_table = Table(list(subhalo_grp.values()), names=(subhalo_keys))
subhalo_table.add_column([x for x in range(len(subhalo_table))], name='Index' ,index=0)

del subhalo_grp

halo_table = Table(list(halo_grp.values()), names=(halo_keys))
halo_table.add_column([x for x in range(len(halo_table))], name='SubhaloGrNr' ,index=0)

subhalo_table = join(subhalo_table, halo_table, keys='SubhaloGrNr', join_type='left')

subhalo_table.add_column(subhalo_table["SubhaloMassType"][:,4], name=str("StellarMass"))
subhalo_table.add_column(subhalo_table['GroupFirstSub'] == subhalo_table['Index'],name=str("IsCentral"))

17120875
19595267


In [16]:
del halo_grp
del halo_table

In [17]:
def cut_color_lines(table, min_gmr, max_gmr):
    mag = table['SubhaloStellarPhotometrics']
    gmr_cut_low = (mag[:,4]-mag[:,5]) > min_gmr
    gmr_cut_high = (mag[:,4]-mag[:,5])< max_gmr
    return  table[gmr_cut_low & gmr_cut_high]

def cut_mass_high(table, field, num):
    if(field == "SubhaloMassType" or field == "SubhaloMassInRadType"):
        subhalomass = np.array(np.log10(table[field][:,4]*1e10))
        mask = subhalomass > num
        return table[mask]
    else:
        subhalomass = np.array(np.log10(table[field]*1e10))
        mask = subhalomass > num
        return table[mask]

def cut_mass_low(table, field, num):
    if(field == "SubhaloMassType" or field == "SubhaloMassInRadType"):
        subhalomass = np.array(np.log10(table[field][:,4]*1e10))
        mask = subhalomass < num
        return table[mask]
    else:
        subhalomass = np.array(np.log10(table[field]*1e10))
        mask = subhalomass < num
        return table[mask]

In [18]:
subhalo_table = subhalo_table[subhalo_table['SubhaloFlag']]
subhalo_table = subhalo_table[subhalo_table['SubhaloMassType'][:,4]!=0]

sh81 = cut_mass_high(subhalo_table, "SubhaloMassType", 8)
sh81.write("Subhalo_stellar_8_snapshot"+str(snapshot_ind)+".fits", format='fits')

sh91 = cut_mass_high(subhalo_table, "SubhaloMassType", 9)
sh91.write("Subhalo_stellar_9_snapshot"+str(snapshot_ind)+".fits", format='fits')

sh82 = cut_mass_high(subhalo_table, "SubhaloMass", 8)
sh82.write("Subhalo_total_8_snapshot"+str(snapshot_ind)+".fits", format='fits')

sh92 = cut_mass_high(subhalo_table, "SubhaloMass", 9)
sh92.write("Subhalo_total_9_snapshot"+str(snapshot_ind)+".fits", format='fits')  

In [19]:
cut_mass_high(sh81, "Group_M_Mean200", 13.5).write("SH_stellar8_GM200_13_5_snapshot"+str(snapshot_ind)+".fits",format='fits')  
cut_mass_high(sh81, "Group_M_Mean200", 13.7).write("SH_stellar8_GM200_13_7_snapshot"+str(snapshot_ind)+".fits",format='fits')  
cut_mass_high(sh81, "Group_M_Mean200", 14).write("SH_stellar8_GM200_14_snapshot"+str(snapshot_ind)+".fits",format='fits')  

cut_mass_high(sh82, "Group_M_Mean200", 13.5).write("SH_total8_GM200_13_5_snapshot"+str(snapshot_ind)+".fits",format='fits')
cut_mass_high(sh82, "Group_M_Mean200", 13.7).write("SH_total8_GM200_13_7_snapshot"+str(snapshot_ind)+".fits",format='fits')  
cut_mass_high(sh82, "Group_M_Mean200", 14).write("SH_total8_GM200_14_snapshot"+str(snapshot_ind)+".fits",format='fits')  

cut_mass_high(sh91, "Group_M_Mean200", 13.5).write("SH_stellar9_GM200_13_5_snapshot"+str(snapshot_ind)+".fits",format='fits')
cut_mass_high(sh91, "Group_M_Mean200", 13.7).write("SH_stellar9_GM200_13_7_snapshot"+str(snapshot_ind)+".fits",format='fits')  
cut_mass_high(sh91, "Group_M_Mean200", 14).write("SH_sterllar9_GM200_14_snapshot"+str(snapshot_ind)+".fits",format='fits')  

cut_mass_high(sh92, "Group_M_Mean200", 13.5).write("SH_total9_GM200_13_5_snapshot"+str(snapshot_ind)+".fits",format='fits')
cut_mass_high(sh92, "Group_M_Mean200", 13.7).write("SH_total9_GM200_13_7_snapshot"+str(snapshot_ind)+".fits",format='fits')  
cut_mass_high(sh92, "Group_M_Mean200", 14).write("SH_total9_GM200_14_snapshot"+str(snapshot_ind)+".fits",format='fits')  


<ipython-input-17-8970e05913a2>:13: RuntimeWarning: divide by zero encountered in log10
  subhalomass = np.array(np.log10(table[field]*1e10))
